In [10]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from urllib.parse import urljoin
import re
import time
import threading

In [11]:
driver =  webdriver.Chrome(executable_path = 'D:\workspace\chromedriver.exe' )

driver.get('https://ev.or.kr/evmonitor')
#time.sleep(5)

In [12]:
sid = []
name =[]  #이름
add = []  #주소
u_time =[]  #사용가능시간
typ = []  # 구분
agency = [] # 운영기관
phone = [] # 연락처
fee = [] # 충전요금
amount = [] # 급속충전량
etc = [] # 참고사항
nouse = [] # 운전상태
using = []
ben = []
charge = []

In [6]:
def get_id():
    global sid
    for k in range(2,19):
        driver.find_element_by_css_selector('#F_SI_DO1')
        driver.implicitly_wait(5)
        driver.find_element_by_css_selector('#F_SI_DO1 > option:nth-child(%s)'%k).click()
        #time.sleep(3)
        driver.find_element_by_css_selector('#statList')
        driver.implicitly_wait(30)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.find_all('div',class_='fw_path')
        for link in a:
            temp = str(link['sid'])
            sid.append(temp)

In [13]:
def get_data():
    global name
    global add
    global time
    global typ
    global agency
    global phone
    global fee
    global amount
    global etc
    global nouse
    global using
    global ben
    global charge
    
    for i in sid[0:20]:
        url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
        url = url_base+str(i)
        html = urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')

        n_tmp = soup.find_all('h2')
        name.append(n_tmp[0].get_text())

        t_tmp = soup.find_all('em')
        u_time.append(t_tmp[0].get_text())

        ty_tmp = soup.find_all('dd')
        typ.append(ty_tmp[0].get_text())

        info_tmp = soup.find_all(class_='table_02')
        dinfo = info_tmp[0].find_all('td')
        add.append(dinfo[0].get_text())
        agency.append(dinfo[1].get_text())
        phone.append(dinfo[2].get_text())
        tmp_fee = dinfo[3].get_text()
        tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
        fee.append(tmp_fee)
        tmp_amount = dinfo[4].get_text()
        tmp_amount = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_amount)
        amount.append(tmp_amount)
        tmp_etc = dinfo[5].get_text()
        tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
        etc.append(tmp_etc)
        no=0
        use=0
        den=0
        for tmp_using in soup.find_all(class_='td3'):
            a = tmp_using.find_all('span')
            tmp = a[0].get_text()
            if tmp =='사용가능':
                no +=1
            elif tmp == '사용중':
                use +=1
            else:
                den +=1
        nouse.append(no)
        using.append(use)
        ben.append(den)
        
    td2 = soup.find_all(class_='td2')
    for te in td2[0:]:
        test=[]
        for a in te.find_all('span'):
            if a.get_text()=='사용가능':
                break
            elif a.get_text()=='사용중':
                break
            elif a.get_text()=='사용불가':
                break
            elif a.get_text()==' ':
                break
            elif len(a.get_text())<6:
                tmp=[]
                tmp.append(a.get_text())
                #print(tmp)
                test+=tmp
        test= str("+".join(test))
        charge.append(test)
        if test!=None:
            break

    t04 = soup.find_all(class_='ev_type t04')
    for te in t04[1:]:
        test=[]
        for b in te.find_all('span'):
            if len(b.get_text())<6:
                tmp=[]
                tmp.append(b.get_text())
                test+=tmp
        test= str("+".join(test))
        #print(test)
        charge.append(test)
        if test!=None:
            break


In [14]:
import time
def thread_run():
    print('====',time.ctime(),'====')
    get_id()
    print('id_finish')
    threading.Timer(86400, thread_run).start()
thread_run()
    
def data_thread_run():
    print('====',time.ctime(),'====')
    for i in range(1,50001):
        name =[]  #이름
        add = []  #주소
        u_time =[]  #사용가능시간
        typ = []  # 구분
        agency = [] # 운영기관
        phone = [] # 연락처
        fee = [] # 충전요금
        amount = [] # 급속충전량
        etc = [] # 참고사항
        nouse = [] # 운전상태
        using = []
        ben = []
        charge = []
        for i in sid[0:10]:
            url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
            url = url_base+str(i)
            html = urlopen(url)
            soup = BeautifulSoup(html, 'html.parser')

            n_tmp = soup.find_all('h2')
            name.append(n_tmp[0].get_text())

            t_tmp = soup.find_all('em')
            u_time.append(t_tmp[0].get_text())

            ty_tmp = soup.find_all('dd')
            typ.append(ty_tmp[0].get_text())

            info_tmp = soup.find_all(class_='table_02')
            dinfo = info_tmp[0].find_all('td')
            add.append(dinfo[0].get_text())
            agency.append(dinfo[1].get_text())
            phone.append(dinfo[2].get_text())
            tmp_fee = dinfo[3].get_text()
            tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
            fee.append(tmp_fee)
            tmp_amount = dinfo[4].get_text()
            tmp_amount = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_amount)
            amount.append(tmp_amount)
            tmp_etc = dinfo[5].get_text()
            tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
            etc.append(tmp_etc)
            no=0
            use=0
            den=0
            for tmp_using in soup.find_all(class_='td3'):
                a = tmp_using.find_all('span')
                tmp = a[0].get_text()
                if tmp =='사용가능':
                    no +=1
                elif tmp == '사용중':
                    use +=1
                else:
                    den +=1
            nouse.append(no)
            using.append(use)
            ben.append(den)

        td2 = soup.find_all(class_='td2')
        for te in td2[0:]:
            test=[]
            for a in te.find_all('span'):
                if a.get_text()=='사용가능':
                    break
                elif a.get_text()=='사용중':
                    break
                elif a.get_text()=='사용불가':
                    break
                elif a.get_text()==' ':
                    break
                elif len(a.get_text())<6:
                    tmp=[]
                    tmp.append(a.get_text())
                    #print(tmp)
                    test+=tmp
            test= str("+".join(test))
            charge.append(test)
            if test!=None:
                break

        t04 = soup.find_all(class_='ev_type t04')
        for te in t04[1:]:
            test=[]
            for b in te.find_all('span'):
                if len(b.get_text())<6:
                    tmp=[]
                    tmp.append(b.get_text())
                    test+=tmp
            test= str("+".join(test))
            #print(test)
            charge.append(test)
            if test!=None:
                break
        print('data_finish')
        print(name)
    threading.Timer(20, thread_run).start()
data_thread_run()

==== Wed Sep  2 15:23:31 2020 ====
id_finish
==== Wed Sep  2 15:23:42 2020 ====
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장'

KeyboardInterrupt: 

In [ ]:
from datetime import datetime
while True:
    now = datatime.now()
    print(now)
    get_data()
    print('data_finish')
    print(name, using)
    

In [ ]:
def data_frame():
    df = pd.DataFrame({'이름':name, '사용가능시간':u_time,'구분':typ, '사용가능':nouse,'사용중':using,'사용불가':ben,'도로명주소':add,'연락처':phone,'충전요금':fee, '급속충전량':amount,'참고사항':etc})
    df.head()